<a href="https://colab.research.google.com/github/KeeganMcWW/ml-projects/blob/main/IASIM2016_Challenege_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and set up the Colab

In [1]:
import scipy.io as sio
import os
import matplotlib.pyplot as plt
import numpy as np

from google.colab import files
from scipy.signal import savgol_filter
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

In [2]:
%matplotlib inline

## Fetch and store data from Eiganvector.com

In [3]:
machine_data_path = r'/content/data_files/'
try:
    os.mkdir(machine_data_path)
except OSError:
      print ("Creation of the directory %s failed" % machine_data_path)
else:
    print ("Successfully created the directory %s " % machine_data_path)

Creation of the directory /content/data_files/ failed


In [ ]:
!wget https://eigenvector.com/wp-content/uploads/2021/03/IASIM16ChallengeProblem.zip

--2022-01-12 20:53:28--  https://eigenvector.com/wp-content/uploads/2021/03/IASIM16ChallengeProblem.zip
Resolving eigenvector.com (eigenvector.com)... 69.163.163.60, 2607:f298:6:a034::eaf:812c
Connecting to eigenvector.com (eigenvector.com)|69.163.163.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 448641963 (428M) [application/zip]
Saving to: ‘IASIM16ChallengeProblem.zip.1’

allengeProblem.zip.  67%[============>       ] 288.60M  21.0MB/s    eta 9s     

In [ ]:
!unzip IASIM16ChallengeProblem.zip -d data_files

## Helper functions to parse the .mat files for use in python

In [ ]:
# https://stackoverflow.com/questions/7008608/scipy-io-loadmat-nested-structures-i-e-dictionaries
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], sio.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, sio.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, sio.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = sio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

#Examining the Data

load the imported data

In [ ]:
wheat_gluten_pure = loadmat('/content/data_files/Wheat_Gluten_Pure.mat')
melamine_pure = loadmat('/content/data_files/Melamine_Pure.mat')
m_200ppm = loadmat('/content/data_files/M_200ppm.mat')

Data is imported with as a dict with nestled dicts. The important dict is the 'z' dictionary, which contains the important information.

In [ ]:
print(wheat_gluten_pure['z'].keys())

Here we can see the data is imported to the correct size. The hyperspectral image should be 243 by 244 with 229 spectral points (a 243x244x229 HS cube).

In [ ]:
x_size, y_size = wheat_gluten_pure['z']['imagesize']
z_size = np.shape(wheat_gluten_pure['z']['data'])[1]
print(x_size, y_size, z_size)

In [ ]:
wheat_gluten_pure_data_mat = np.reshape(wheat_gluten_pure['z']['data'], (x_size, y_size, z_size), order = 'F')
melamine_pure_data_mat = np.reshape(melamine_pure['z']['data'], (x_size, y_size, z_size), order = 'F')
m_200ppm_data_mat = np.reshape(m_200ppm['z']['data'], (x_size, y_size, z_size), order = 'F')

lets take a look at the image. Here I am just slicing along the spectral (z) axis. Each pixel is just the reflectance intensity at the given wavelength. For example, when z_slice = 0 we are looking at the reflectance instensity at 1120nm for the image. 

Something important to note is it looks like there is some non uniformity in these "pure" samples. This problem of typical for any given sampling problem. We have to be careful that by naively training these "pure" spectra we don't create outliers which skew the model. 

In [ ]:
z_slice = 101 #@param {type: "slider", min: 0, max: 228}
fig, ax = plt.subplots(1,3, figsize=(10, 10));
plt.tight_layout();
ax[0].imshow(wheat_gluten_pure_data_mat[:, :, z_slice]);
ax[0].set_title('Pure Wheat Gluten');
ax[1].imshow(melamine_pure_data_mat[:, :, z_slice]);
ax[1].set_title('Pure Melamine');
ax[2].imshow(m_200ppm_data_mat[:, :, z_slice]);
ax[2].set_title('200ppm of Malamin');

No we can slice along one of the image axis (either x or y) to check out the spectral data. Here I am slicing along the x axis. 

In [ ]:
x_slice = 79 #@param {type: "slider", min: 0, max: 242}
fig, ax = plt.subplots(3,1, figsize=(8,8));
plt.tight_layout()
ax[0].plot(wheat_gluten_pure_data_mat[x_slice, :, :].T)
ax[0].set_title('Pure Wheat Gluten');
ax[1].plot(melamine_pure_data_mat[x_slice, :, :].T)
ax[1].set_title('Pure Melamine');
ax[2].plot(m_200ppm_data_mat[x_slice, :, :].T)
ax[2].set_title('200ppm of Malamin');
plt.show()

Now we can pull the center(ish) pixel from each of the three training HSIs (Hyper Spectral Images) just to take a look at them. The pure melamine spectrum is pretty prenounced, however the low concentration melamine sample clearly has the backgroup spectrim of thw wheat gluten dominating. This is typical for in "target detection", but makes for a challenging problem. 

In [ ]:
fig, ax = plt.subplots()
ax.plot(wheat_gluten_pure_data_mat[122, 122, :].T, label = 'Pure Wheat Gluten')
ax.plot(melamine_pure_data_mat[122, 122, :].T, label = 'Pure Melamine')
ax.plot(m_200ppm_data_mat[122, 122, :].T, label = '200ppm Melamine')
ax.legend()
plt.show()

Right now I am just cutting a 100x100 square out of each image. This really should be optimized for the full shape, but this is quick.

We can then view it by again slicing along the spectral axis. 

In [ ]:
crop_size = 100
left = int(np.floor((x_size - crop_size)/2))
top = int(np.floor((y_size - crop_size)/2))
right = int(np.floor((x_size + crop_size)/2))
bottom = int(np.floor((y_size + crop_size)/2))

wheat_cropped = wheat_gluten_pure_data_mat[left:right, top:bottom, :]
melamine_cropped = melamine_pure_data_mat[left:right, top:bottom, :]
m_200ppm_cropped = m_200ppm_data_mat[left:right, top:bottom, :]

In [ ]:
z_slice = 34 #@param {type: "slider", min: 0, max: 242}
fig, ax = plt.subplots(1,3, figsize=(10, 10));
plt.tight_layout();
ax[0].imshow(wheat_cropped[:, :, z_slice]);
ax[0].set_title('Pure Wheat Gluten');
ax[1].imshow(melamine_cropped[:, :, z_slice]);
ax[1].set_title('Pure Melamine');
ax[2].imshow(m_200ppm_cropped[:, :, z_slice]);
ax[2].set_title('200ppm of Malamin');

# Preprocessing the Data

## Savitzky-Golay filter

In [ ]:
def apply_sgfilter(matrix, window_length, polyorder, deriv):
  output = np.empty(np.shape(matrix))
  for i in range(np.shape(matrix)[0]):
    for j in range(np.shape(matrix)[1]):
      output[i][j] = savgol_filter(matrix[i, j, :], window_length, polyorder, deriv=deriv)
      #output[i][j] = np.gradient(sg_filter)
  return output

First, we are going to test a few smoothing parameters. We will be testing a few different window size/polynomial ratios. The NIRPyResearch articles below give a very good explination of this process, the code is mainly inspired by (or copied from) there.

In [ ]:
# https://nirpyresearch.com/choosing-optimal-parameters-savitzky-golay-smoothing-filter/
# https://nirpyresearch.com/savitzky-golay-smoothing-method/
# Set some reasonable parameters to start with
w = 5
p = 2
# Calculate three different smoothed spectra
# first derivative
melamine_sgfilter_1_d1 = apply_sgfilter(melamine_cropped, w, polyorder = p, deriv=1)
melamine_sgfilter_2_d1 = apply_sgfilter(melamine_cropped, 2*w+1, polyorder = p, deriv=1)
melamine_sgfilter_3_d1 = apply_sgfilter(melamine_cropped, 4*w+1, polyorder = 3*p, deriv=1)
# second derivative
melamine_sgfilter_1_d2 = apply_sgfilter(melamine_cropped, w, polyorder = p, deriv=2)
melamine_sgfilter_2_d2 = apply_sgfilter(melamine_cropped, 2*w+1, polyorder = p, deriv=2)
melamine_sgfilter_3_d3 = apply_sgfilter(melamine_cropped, 4*w+1, polyorder = 3*p, deriv=2)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(8,4))
plt.tight_layout()
axs[0].plot(melamine_sgfilter_1_d1[50, :, :][0], label = 'Smoothing: w/p = 2.5')
axs[0].plot(melamine_sgfilter_2_d1[50, :, :][0], label = 'Smoothing: w/p = 5.5')
axs[0].plot(melamine_sgfilter_3_d1[50, :, :][0], label = 'Smoothing: w/p = 3.5')
axs[0].set_title('Pure Melamine, First Derivative');
axs[1].plot(melamine_sgfilter_1_d2[50, :, :][0], label = 'Smoothing: w/p = 2.5')
axs[1].plot(melamine_sgfilter_2_d2[50, :, :][0], label = 'Smoothing: w/p = 5.5')
axs[1].plot(melamine_sgfilter_3_d3[50, :, :][0], label = 'Smoothing: w/p = 3.5')
axs[1].set_title('Pure Melamine, Second Derivative');
plt.legend()
plt.show()

In [ ]:
wheat_preprocessed = apply_sgfilter(wheat_cropped, window_length = w, polyorder = p, deriv=2)
meamine_preprocessed = apply_sgfilter(melamine_cropped, window_length = w, polyorder = p, deriv=2)
m_200ppm_preprocessed = apply_sgfilter(m_200ppm_cropped, window_length = w, polyorder = p, deriv=2)
dataset_preprocessing = np.concatenate((wheat_preprocessed, meamine_preprocessed, m_200ppm_preprocessed))

In [ ]:
x_slice = 43 #@param {type: "slider", min: 0, max: 99}
fig, axs = plt.subplots(3, 2, figsize=(8,8))
plt.tight_layout()
axs[0, 0].plot(wheat_cropped[x_slice, :, :].T)
axs[0, 0].set_title('Pure Wheat Gluten');
axs[1, 0].plot(melamine_cropped[x_slice, :, :].T)
axs[1, 0].set_title('Pure Melamine');
axs[2, 0].plot(m_200ppm_cropped[x_slice, :, :].T)
axs[2, 0].set_title('200ppm of Malamin');
axs[0, 1].plot(wheat_preprocessed[x_slice, :, :].T)
axs[0, 1].set_title('Pure Wheat Gluten');
axs[1, 1].plot(meamine_preprocessed[x_slice, :, :].T)
axs[1, 1].set_title('Pure Melamine');
axs[2, 1].plot(m_200ppm_preprocessed[x_slice, :, :].T)
axs[2, 1].set_title('200ppm of Malamin');
plt.show()

Here I am going to great a few labels for a supervised learning algorthem. 

In [ ]:
def create_label(matrix, label):
  output = np.empty(np.shape(matrix[:,:,-1]))
  for i in range(np.shape(matrix)[0]):
    for j in range(np.shape(matrix)[1]):
      output[i][j] = label
  return output

In [ ]:
wheat_labels = create_label(wheat_cropped, 1)
melamine_labels = create_label(melamine_cropped, 0)
#m_200ppm_labels = create_label(m_200ppm_preprocessed, 200/1000000)
m_200ppm_labels = create_label(m_200ppm_cropped, 0)
wheat_labels_flat = np.reshape(wheat_labels, (crop_size * crop_size), order = 'F')
melamine_labels_flat = np.reshape(melamine_labels, (crop_size * crop_size), order = 'F')
m_200ppm_labels_flat = np.reshape(m_200ppm_labels, (crop_size * crop_size), order = 'F')

In [ ]:
wheat_flat = np.reshape(wheat_cropped, (crop_size * crop_size, z_size), order = 'F')
melamine_flat = np.reshape(melamine_cropped, (crop_size * crop_size, z_size), order = 'F')
m_200ppm_flat = np.reshape(m_200ppm_cropped, (crop_size * crop_size, z_size), order = 'F')

In [ ]:
dataset = np.concatenate((wheat_flat, melamine_flat, m_200ppm_flat))
labels = np.concatenate((wheat_labels_flat, melamine_labels_flat, m_200ppm_labels_flat))

In [ ]:
wheat_preprocessed_labels = create_label(wheat_preprocessed, 1)
melamine_preprocessed_labels = create_label(meamine_preprocessed, 0)
#m_200ppm_labels = create_label(m_200ppm_preprocessed, 200/1000000)
m_200ppm_preprocessed_labels = create_label(m_200ppm_preprocessed, 0)
wheat_preprocessed_labels_flat = np.reshape(wheat_preprocessed_labels, (crop_size * crop_size), order = 'F')
melamine_preprocessed_labels_flat = np.reshape(melamine_preprocessed_labels, (crop_size * crop_size), order = 'F')
m_200ppm_preprocessed_labels_flat = np.reshape(m_200ppm_preprocessed_labels, (crop_size * crop_size), order = 'F')

In [ ]:
wheat_preprocessed_flat = np.reshape(wheat_preprocessed, (crop_size * crop_size, z_size), order = 'F')
meamine_preprocessed_flat = np.reshape(meamine_preprocessed, (crop_size * crop_size, z_size), order = 'F')
m_200ppm_preprocessed_flat = np.reshape(m_200ppm_preprocessed, (crop_size * crop_size, z_size), order = 'F')

In [ ]:
dataset_preprocessed = np.concatenate((wheat_preprocessed_flat, meamine_preprocessed_flat, m_200ppm_preprocessed_flat))
labels_preprocessed = np.concatenate((wheat_preprocessed_labels_flat, melamine_preprocessed_labels_flat, m_200ppm_preprocessed_labels_flat))

In [ ]:
print(np.shape(dataset))
print(np.shape(labels))

In [ ]:
print(np.shape(dataset_preprocessed))
print(np.shape(labels_preprocessed))

## PCA Decomposition


Here I am letting the PCA algorthem decompose the spectrum in some number of principle components based on some describable variance criteria. That is to say, n compoenents will be solved until some percetage of variance is explained by the PCA decompisition.  

In [ ]:
pca = PCA(n_components=0.98, svd_solver = 'full')
pca.fit(dataset)
print(pca)
print('The number of components is: ' + str(len(pca.explained_variance_ratio_)))

In [ ]:
pca_preprocessed = PCA(n_components=0.98, svd_solver = 'full')
pca_preprocessed.fit(dataset_preprocessed)
print(pca_preprocessed)
print('The number of components is: ' + str(len(pca_preprocessed.explained_variance_ratio_)))

We can graph the variance explained by each additional PC. for this problem a large amount if variance can be explained by a single PC. This is typical for spectral data since it is highly correlated. 

This is handy since now we have a means to transform our large number of corrilated spectral points into a number of uncorrilated features (in the form of orthogonal PCs). 

In [ ]:
PCs_to_view = 10
fig, ax = plt.subplots()
plt.plot(pca.explained_variance_ratio_)
ax.set(xlabel='Princple Components', ylabel='Explained Variance',)
plt.tight_layout();
ax.grid()
ax.xaxis
for x,y in zip(range(PCs_to_view),pca.explained_variance_ratio_):

    label = "{:.3f}".format(y)

    plt.annotate(label, # this is the text
                 (x,y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(0,3), # distance from text to points (x,y)
                 ha='left') # horizontal alignment can be left, right or center
plt.xlim(0,PCs_to_view)
plt.title('Explained Variance per Principle Component')
plt.show()

# Data Pipeline and ML Model

I am going to give a SVM a shot at labeling outliers. The idea is that the spectral preprocessing and decompisiton give the SVM the oppertunity to be able to descriminate the low concetration melamine spectra from background wheat gluten. 

We can tune the C parameter with a hyperparameter tuner, we well as play with other options like the kernal type. At this point it is worth pointing out we have other parameters to consider as well (for example the SG Filters parameters). We will start with the model parameters to keep things simple. 

In [ ]:
model = svm.SVC()
#model.fit(dataset_transformed, labels)

I want to try to see how the stock sklearn SVM does on both the preprocessed data  as well as the unprocessed data.

In [ ]:
#%%timeit
# the timeit reports a value of ~5 min 20 sec. for the unpreprocessed data
cv = KFold(n_splits=5, random_state=1, shuffle=True)
scores = cross_val_score(model, dataset, labels, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

In [ ]:
#%%timeit
# the timeit reports a value of ~5 min 20 sec. for the unpreprocessed data
cv = KFold(n_splits=5, random_state=1, shuffle=True)
scores = cross_val_score(model, dataset_preprocessed, labels_preprocessed, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Now we can train an actual svm for use on one of the test examples. 

In [ ]:
val_split = 0.3
X_train, X_test, y_train, y_test = train_test_split(dataset_preprocessed, labels_preprocessed, test_size=val_split, random_state=0, shuffle = True)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
def predict(matrix, model):
  output = np.empty(np.shape(matrix[:,:,-1])) 
  preprocessed = apply_sgfilter(matrix, w, p, 1)
  for i in range(np.shape(matrix)[0]):
    for j in range(np.shape(matrix)[1]):
      #pca_transform = pca.transform(preprocessed[i,j])
      output[i][j] = model.predict(preprocessed[i,j].reshape(1, -1))
  return output

In [ ]:
test_1 = loadmat('/content/data_files/Test_1.mat')
test_1_mat = np.reshape(test_1['z']['data'], (x_size, y_size, z_size), order = 'F')
test_1_predict = predict(test_1_mat, model)

In [ ]:
plt.imshow(test_1_predict)

In [ ]:
test_2 = loadmat('/content/data_files/Test_2.mat')
test_2_mat = np.reshape(test_2['z']['data'], (x_size, y_size, z_size), order = 'F')
test_2_predict = predict(test_2_mat, model)

In [ ]:
plt.imshow(test_2_predict)

In [ ]:
train_check = loadmat('/content/data_files/Wheat_Gluten_Pure.mat')
train_check_mat = np.reshape(train_check['z']['data'], (x_size, y_size, z_size), order = 'F')
train_check_mat_predict = predict(train_check_mat, model)

In [ ]:
plt.imshow(train_check_mat_predict)